In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('localhost', 27017)

db = client.companies_v1
print(db)


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies_v1')


In [2]:

oficina = db.companies_v1.find({})
    #{'$and':
                            #  [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            ##   { "offices.longitude": {"$exists": True,"$ne":None} },
                            #   {'founded_year': {'$gt': 2005 }}, 
                            #   {'number_of_employees': {'$gt': 1}}, 
                            #   {'$or':[{'category_code': 'web' },
                                    #   {'category_code': 'software' },
                                    #   {'category_code': 'games_video' },
                                    #   {'category_code': 'mobile' },
                                    #   {'category_code': 'music' },
                                     #  {'category_code': 'nanotech'}]}]}, 
                           #  {'name':1,'founded_year': 1, '_id': 0, 'number_of_employees':1,'offices.latitude':1, 'offices.longitude':1, 'category_code':1})

dataset1 = pd.DataFrame(oficina)
dataset1.head()

,_id,category_code,founded_year,geojson,latitude,longitude,name,number_of_employees,offices
0,5cd5954ae217c00db4dcb3ac,web,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,"[{'longitude': -118.393064, 'latitude': 34.090..."
1,5cd5954ae217c00db4dcb3ad,web,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Mahalo,40,"[{'longitude': -118.487267, 'latitude': 34.017..."
2,5cd5954ae217c00db4dcb3ae,games_video,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,Kyte,40,"[{'longitude': -122.409173, 'latitude': 37.788..."
3,5cd5954ae217c00db4dcb3af,web,2007,"{'type': 'Point', 'coordinates': [-119.306607,...",37.269175,-119.306607,CriticalMetrics,4,"[{'longitude': -119.306607, 'latitude': 37.269..."
4,5cd5954ae217c00db4dcb3b0,games_video,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,"[{'longitude': -118.254558, 'latitude': 34.051..."


In [3]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 9 columns):
_id                    1498 non-null object
category_code          1498 non-null object
founded_year           1498 non-null int64
geojson                1498 non-null object
latitude               1498 non-null float64
longitude              1498 non-null float64
name                   1498 non-null object
number_of_employees    1498 non-null int64
offices                1498 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 105.4+ KB


In [8]:

nearLocation = {
    "lng": 37.793568,
    "lat": -122.404621
}

dataset1 = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 600, # In meters
     }
   }
}))
sum(dataset1.number_of_employees)

640

In [9]:
lista=[]
for a,b in zip(dataset1['longitude'], dataset1['latitude']):
    numero_emp_cerca = pd.DataFrame(db.companies_v1.find({
    "geojson": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ a , b ]
       },
       "$maxDistance": 600, # In meters
     }
   }
}))
    sum(dataset1.number_of_employees)
    lista.append(numero_emp_cerca)
print(len(lista))

18


In [6]:
dataset1["empleados_cerca"]=lista
dataset1.sort_values(["empleados_cerca"], ascending=[False], inplace=True) 
dataset1.info()

ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:


#def numero_emp_cerca(geojson, max_distance = 600):
  #  emp_cerca = pd.DataFrame(list(companies_v1.find(
      #  {'geojson': { '$near': {'$geometry': geojson, '$maxDistance': max_distance}}})))
    #return sum(emp_cerca['number_of_employees'])


#data['employees2000'] = data['geojson'].apply(numero_emp_cerca)
#data.head()


